# NOTE
This file should be executed only after both .h5 files - trained_densenet_model.h5 and trained_inception_model.h5 are created.
I would recommend running this file after evaluating the performance of the individual models.

We have seen the performance of the individual models by running inception_run.py and densenet_run.py separately. If you want to
see a screenshot, navigate to images/densenet_perf.png or images/inception.png.

By using ensemble method - weighted averaging, we can produce a combined model that may show better performance compared to the 
individual models.

To start, let us import the libraries:

In [ ]:
import keras
import numpy as np
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import load_model
from sklearn.model_selection import train_test_split

In [ ]:
x_test = np.load("testx.npy")
y_test = np.load("testy.npy")

In [ ]:
num_classes = 3

x_test = x_test.reshape(-1, 512, 512, 3)
x_test = x_test.astype('float32')
x_test = x_test / 255.

# Converted to one-hot encoding
y_test = to_categorical(y_test, num_classes)

We load each model and store it in an array called members.

In [ ]:
members = [load_model("trained_densenet_model.h5"), load_model("trained_inception_model.h5")]

# TESTING

Each model in members makes predictions on the input data - x_test.
Then their predictions are stored in predictions array.
predictions[0] -> Predictions on x_test by densenet
predictions[1] -> Predictions on x_test by inceptionV3

In [ ]:
predictions = [model.predict(x_test) for model in members]
predictions = np.array(predictions)

Now, since inceptionV3 has shown better performance as compared to densenet, we give higher weightage to predictions made
by inceptionV3.

In [ ]:
predictions[0] = predictions[0] * 0.3
predictions[1] = predictions[1] * 0.7

Summing the predictions

In [ ]:
summed = np.sum(predictions, axis=0)

Converted predictions and actual outputs to integer from one-hot representation

In [ ]:
predicted_classes = np.argmax(np.round(summed), axis=1)
y_actual_classes = np.argmax(np.round(y_test), axis=1)  # Converted one-hot to integer

Prints number of correct and incorrect predictions, respectively.

In [ ]:
correct = np.where(predicted_classes == y_actual_classes)[0]
print("Found " + str(len(correct)) + " correct labels")

incorrect = np.where(predicted_classes != y_actual_classes)[0]
print("Found " + str(len(incorrect)) + " incorrect labels")

Prints confusion matrix and classification report.

In [ ]:
print("CONFUSION MATRIX")
print(confusion_matrix(y_true=y_actual_classes, y_pred=predicted_classes))

print("CLASSIFICATION REPORT")
target_names = ["COVID-19", "Normal", "Viral Pneumonia"]
print(classification_report(y_actual_classes, predicted_classes, target_names=target_names, digits = 5))